In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from qutip import *

# Linear inversion

In [59]:
import numpy as np

M1 = 0.5 * np.array([[2, -(1-1j), -(1+1j), 1],
                     [-(1+1j), 0, 1j, 0],
                     [-(1-1j), -1j, 0, 0],
                     [1, 0, 0, 0]]) #cite: 18

M2 = 0.5 * np.array([[0, -(1-1j), 0, 1],
                     [-(1+1j), 2, 1j, -(1+1j)],
                     [0, -1j, 0, 0],
                     [1, -(1+1j), 0, 0]]) #cite: 18

M3 = 0.5 * np.array([[0, 0, 0, 1],
                     [0, 0, 1j, -(1+1j)],
                     [0, -1j, 0, -(1-1j)],
                     [1, -(1-1j), -(1+1j), 2]]) #cite: 18

M4 = 0.5 * np.array([[0, 0, -(1+1j), 1],
                     [0, 0, 1j, 0],
                     [-(1-1j), -1j, 2, -(1-1j)],
                     [1, 0, -(1+1j), 0]]) #cite: 18

M5 = 0.5 * np.array([[0, 0, 2j, -(1+1j)],
                     [0, 0, (1-1j), 0],
                     [-2j, (1+1j), 0, 0],
                     [-(1-1j), 0, 0, 0]]) #cite: 18

M6 = 0.5 * np.array([[0, 0, 0, -(1+1j)],
                     [0, 0, (1-1j), 2j],
                     [0, (1+1j), 0, 0],
                     [-(1-1j), -2j, 0, 0]]) #cite: 19

M7 = 0.5 * np.array([[0, 0, 0, -(1+1j)],
                     [0, 0, -(1-1j), 2],
                     [0, -(1+1j), 0, 0],
                     [-(1-1j), 2, 0, 0]]) #cite: 19

M8 = 0.5 * np.array([[0, 0, 2, -(1+1j)],
                     [0, 0, -(1-1j), 0],
                     [2, -(1+1j), 0, 0],
                     [-(1-1j), 0, 0, 0]]) #cite: 2, 6, 7, 9, 10, 11, 12, 13, 14 (Note: This matrix definition appears to be spread across multiple lines in the source, so it has been reconstructed. The last row and column are empty according to the input.)

M9 = np.array([[0, 0, 0, 1j],
               [0, 0, -1j, 0],
               [0, 1j, 0, 0],
               [-1j, 0, 0, 0]]) #cite: 15

M10 = np.array([[0, 0, 0, 1],
                [0, 0, 1, 0],
                [0, 1, 0, 0],
                [1, 0, 0, 0]]) #cite: 15

M11 = np.array([[0, 0, 0, 1j],
                [0, 0, 1j, 0],
                [0, -1j, 0, 0],
                [-1j, 0, 0, 0]]) #cite: 16

M12 = 0.5 * np.array([[0, 2, 0, -(1+1j)],
                      [2, 0, -(1+1j), 0],
                      [0, -(1-1j), 0, 0],
                      [-(1-1j), 0, 0, 0]]) #cite: 18

M13 = 0.5 * np.array([[0, 0, 0, -(1+1j)],
                      [0, 0, -(1+1j), 0],
                      [0, -(1-1j), 0, 2],
                      [-(1-1j), 0, 2, 0]]) #cite: 18

M14 = 0.5 * np.array([[0, 0, 0, -(1-1j)],
                      [0, 0, -(1-1j), 0],
                      [0, -(1+1j), 0, -2j],
                      [-(1+1j), 0, 2j, 0]]) #cite: 18

M15 = 0.5 * np.array([[0, -2j, 0, -(1-1j)],
                      [2j, 0, (1-1j), 0],
                      [0, (1+1j), 0, 0],
                      [-(1+1j), 0, 0, 0]]) #cite: 18

M16 = np.array([[0, 0, 0, 1],
                [0, 0, -1, 0],
                [0, -1, 0, 0],
                [1, 0, 0, 0]]) #cite: 21

matrices = [M1, M2, M3, M4, M5, M6, M7, M8, M9, M10, M11, M12, M13, M14, M15, M16]
qutip_mtrx = [Qobj(M) for M in matrices]

M = dict(zip(range(1, 17), qutip_mtrx))
M1

array([[ 1. +0.j , -0.5+0.5j, -0.5-0.5j,  0.5+0.j ],
       [-0.5-0.5j,  0. +0.j ,  0. +0.5j,  0. +0.j ],
       [-0.5+0.5j,  0. -0.5j,  0. +0.j ,  0. +0.j ],
       [ 0.5+0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ]])

In [60]:
# Phi-, Phi+, Decoherence contain coincidence counts
col_names = ['idx','A', 'B', 'HWPa', 'QWPa', 'HWPb', 'QWPb', 'Phi-', 'Phi+', 'Decoher.']
data = pd.read_excel('Tomography2025.xlsx', header=2, names=col_names, index_col='idx',usecols='A:J', nrows=16)
id_x, id_y = np.where(data.isna())
for id in zip(id_x, id_y):
    data.iloc[id[0], id[1]] = data.iloc[id[0], 6]

data

,A,B,HWPa,QWPa,HWPb,QWPb,Phi-,Phi+,Decoher.
idx,,,,,,,,,
1,H,H,36.0,84,22.0,38,3831,3672.0,18320.0
2,H,V,36.0,84,67.0,38,45,35.0,290.0
3,V,V,81.0,84,67.0,38,3655,3671.0,19310.0
4,V,H,81.0,84,22.0,38,71,75.0,430.0
5,R,H,81.0,129,22.0,38,2218,2218.0,2218.0
6,R,V,81.0,129,67.0,38,1607,1607.0,1607.0
7,D,V,58.5,129,67.0,38,1577,1577.0,1577.0
8,D,H,58.5,129,22.0,38,2219,2219.0,2219.0
9,D,R,58.5,129,22.0,-7,2305,2305.0,2305.0


In [61]:
psi0 = ['Phi-', 'Phi+', 'Decoher.']
densities = dict(zip(psi0, [Qobj(np.zeros((4,4)))]*3))
for state in psi0:
    counts = data[state]
    for i in range(1, 17):
        densities[state] += counts[i]*M[i]
    densities[state] /= np.sum(counts[0:4])

    print(f"Density matrix for {state}:")
    print(densities[state].tidyup())
    print("---------------------------------------------------------------------------------------------")
    print(f"Trace: {densities[state].tr()}")
    print(f"Eigenvalues: {densities[state].eigenenergies()}")
    print("---------------------------------------------------------------------------------------------")
    print("---------------------------------------------------------------------------------------------", '\n')

Density matrix for Phi-:
Quantum object: dims=[[4], [4]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.50394633+0.j         -0.00052618-0.02157327j  0.03525388+0.03512234j
  -0.36700868+0.12825572j]
 [-0.00052618+0.02157327j  0.00591949+0.j         -0.18468824+0.24467245j
  -0.0359116 -0.03196527j]
 [ 0.03525388-0.03512234j -0.18468824-0.24467245j  0.00933965+0.j
  -0.03617469+0.0486714j ]
 [-0.36700868-0.12825572j -0.0359116 +0.02604578j -0.03617469-0.0486714j
   0.48079453+0.j        ]]
---------------------------------------------------------------------------------------------
Trace: (1+0j)
Eigenvalues: [-0.30029286-1.33749051e-04j  0.10270105-5.38157689e-05j
  0.3028792 -1.71078345e-05j  0.8947126 +2.04672655e-04j]
---------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------- 

Density matrix for Phi+:
Quantum object: dims=[[4], [4]

# Maximum likelihood estimation

In [62]:
HH = tensor(basis(2, 0), basis(2, 0))
HV = tensor(basis(2, 0), basis(2, 1))
VH = tensor(basis(2, 1), basis(2, 0))
VV = tensor(basis(2, 1), basis(2, 1))

def a(h, q):
    return 1/np.sqrt(2) * (np.sin(2*h) - 1j*np.sin(2*(h-q)))
def b(h, q):
    return 1/np.sqrt(2) * (np.cos(2*h) + 1j*np.cos(2*(h-q)))

projection_states = []
# A	B	HWPa	QWPa	HWPb	QWPb	Phi-	Phi+	Decoher.
for i in range(16):
    row = data.iloc[i, :]
    projection_states.append(Qobj(
        a(row['HWPa'], row['QWPa'])*a(row['HWPb'], row['QWPb'])*HH +
        a(row['HWPa'], row['QWPa'])*b(row['HWPb'], row['QWPb'])*HV +
        b(row['HWPa'], row['QWPa'])*a(row['HWPb'], row['QWPb'])*VH +
        b(row['HWPa'], row['QWPa'])*b(row['HWPb'], row['QWPb'])*VV))

print(f"|psi_0> = {projection_states[0]}")

def rho_p(t):
    T = Qobj(
        [[t[0], 0, 0, 0],
         [t[4]+1j*t[5], t[1], 0, 0],
         [t[10]+1j*t[11], t[6]+1j*t[7], t[2], 0],
         [t[14]+1j*t[15], t[12]+1j*t[13], t[8]+1j*t[9], t[3]]],
         dims=[[2,2],[2,2]]
    )
    G = T.dag()*T
    return  G/G.tr()

c = [1]*16
rho = rho_p(c)
print(f'Density matrix:\n{rho}')
print(f'Trace: {rho.tr()}')
print(f'Eigenvalues: {rho.eigenenergies()}')

def likelihood(t, measures):
    out = 0
    N = np.sum(measures.iloc[0:4])
    for i in range(16):
        psi = projection_states[i]
        bra_rho_ket = psi.dag() * rho_p(t) * psi
        out += (N*bra_rho_ket - measures.iloc[i])**2 / (2*N*bra_rho_ket)
    if np.imag(out) < 1e-10:
        return np.real(out)
    else:
        raise ValueError("likelihood is very complex ;)")
    
# print(likelihood([1]*16, data['Phi-']))

|psi_0> = Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[-0.26894168+0.07868819j]
 [-0.28337414+0.5975895j ]
 [ 0.04118598-0.26828087j]
 [-0.40828987-0.49365261j]]
Density matrix:
Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.4375+0.j     0.3125-0.0625j 0.1875-0.0625j 0.0625-0.0625j]
 [0.3125+0.0625j 0.3125+0.j     0.1875-0.0625j 0.0625-0.0625j]
 [0.1875+0.0625j 0.1875+0.0625j 0.1875+0.j     0.0625-0.0625j]
 [0.0625+0.0625j 0.0625+0.0625j 0.0625+0.0625j 0.0625+0.j    ]]
Trace: 1.0
Eigenvalues: [0.00460126 0.03507357 0.11137304 0.84895213]


In [81]:
state_name = 'Phi-'
rho0 = densities[state_name]
res = minimize(lambda t: likelihood(t, data[state_name]), np.real(rho0.full().flatten()), method='Powell')
sol = res.x
# print(f'Solution: {sol}')
rho_ = rho_p(sol)
print(f'Eigenvalues: {rho_.eigenenergies()}')
print(f'Tr[⍴] = {(rho_).tr()}')
print(f'Tr[⍴^2] = {(rho_*rho_).tr()}')
rho_.tidyup()

Eigenvalues: [2.61893038e-13 8.80423211e-06 6.85038184e-02 9.31487377e-01]
Tr[⍴] = 0.9999999999999998
Tr[⍴^2] = 0.8723615073418279


Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.27500236+0.j          0.12718496-0.14872091j  0.03659531+0.230863j
  -0.06363239-0.21905981j]
 [ 0.12718496+0.14872091j  0.23285651+0.j         -0.13923328+0.17260689j
   0.10915644-0.21478854j]
 [ 0.03659531-0.230863j   -0.13923328-0.17260689j  0.23180944+0.j
  -0.23799455+0.04081163j]
 [-0.06363239+0.21905981j  0.10915644+0.21478854j -0.23799455-0.04081163j
   0.26033169+0.j        ]]

# Fidelity, Von Neuman entropy, Concurrence

In [82]:
def computeRelevantQuantities(rho, psi0):
    fidelity = psi0.dag() * rho * psi0

    eigenvalues = rho.eigenenergies()
    entropy = -np.sum(eigenvalues*np.log(eigenvalues))   

    concurr = concurrence(rho)

    return np.real(fidelity), entropy, concurr


In [83]:
computeRelevantQuantities(rho_, bell_state('11'))

(0.37156624954816597, 0.24986214540562857, 0.8483098016943369)

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.        ]
 [ 0.70710678]
 [-0.70710678]
 [ 0.        ]]